# Interactive notebook used to prepare code parts for Azure ML

### Test MLFlow SDK 

In [1]:
mlflow_tracking_uri = "azureml://northeurope.api.azureml.ms/mlflow/v1.0/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/resourceGroups/d-azurelm/providers/Microsoft.MachineLearningServices/workspaces/aml-9cxgm0wt"

In [2]:
import mlflow


In [3]:
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [8]:
from mlflow.tracking import MlflowClient

# experiment_name = 'lendingclub-demo-experiment'
# client = MlflowClient()

# exp = mlflow.get_experiment_by_name(experiment_name)
# print(exp)

# runs = mlflow.search_runs(experiment_names=experiment_name)
# print(runs[-1])

experiment_name = 'lendingclub-demo-experiment'
run_id = 'magenta_drawer_9k80sm4w8m'
filter_string = f"tags.mlflow.rootRunId='{run_id}'"
# runs2 = mlflow.search_runs(experiment_names=experiment_name, filter_string=filter_string)
runs = mlflow.search_runs(experiment_names=experiment_name, filter_string=filter_string)
# runs = mlflow.search_runs(experiment_ids=None, search_all_experiments=True, filter_string=filter_string)

# print(finished_mlflow_run)
# metrics = finished_mlflow_run.data.metrics
# tags = finished_mlflow_run.data.tags
# params = finished_mlflow_run.data.params

# print(metrics,tags,params)

In [14]:
print(runs[["metrics.val_accuracy", "tags.estimator_name", "tags.hyperparameters"]].sort_values(by="metrics.val_accuracy", ascending=False))

    metrics.val_accuracy tags.estimator_name     tags.hyperparameters
8               0.887884                None        {"dropout": 0.01}
9               0.887707                None         {"dropout": 0.2}
10              0.887669                None         {"dropout": 0.1}
11              0.887492                None         {"dropout": 0.5}
7               0.804400                None                     None
0                    NaN                None                     None
1                    NaN                None                     None
2                    NaN                None                     None
3                    NaN                None                     None
4                    NaN                None                     None
5                    NaN                None                     None
6                    NaN                None                     None
12                   NaN  LogisticRegression      {"solver": "lbfgs"}
13                  

In [31]:
# Get best model
best_model = runs2[["run_id", "metrics.val_accuracy"]].sort_values(by="metrics.val_accuracy", ascending=False).head(1)
best_run_id = best_model["run_id"].iloc[0]
best_metric = best_model["metrics.val_accuracy"].iloc[0]

print(f"Best run id: {best_run_id}")
print(f"Best metric: {best_metric}")

Best run id: HD_e3040f1e-de19-4348-bcc7-d5cbd78d8236_0
Best metric: 0.8878837823867798


In [32]:
# Register model
model_name = "best_model"
model_path = f"runs:/{best_run_id}/model"
model = mlflow.register_model(model_uri=model_path, name=model_name)

Successfully registered model 'best_model'.
2022/12/16 13:23:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best_model, version 1
Created version '1' of model 'best_model'.


In [33]:
# Get latest version of registered model
client = mlflow.MlflowClient()

model_name = "landing_club"
model_version = client.get_registered_model(model_name).latest_versions[0].version

print(f"Latest version of model {model_name} is {model_version}")

Latest version of model landing_club is 3


'3'

In [34]:
# Deploy model
import json

deploy_config = {
   "instance_type": "Standard_DS2_v2",
   "instance_count": 1,
}

deployment_config_path = "deployment_config.json"
with open(deployment_config_path, "w") as outfile:
   outfile.write(json.dumps(deploy_config))

In [37]:
from mlflow.deployments import get_deploy_client

deploy_client = get_deploy_client(client.tracking_uri)

deploy_client.create_deployment(
   model_uri=f"models:/{model_name}/{model_version}",
   config={ "deploy-config-file": deployment_config_path },
   name=f"landing-club-{model_version}",
)

..................................................................................

{'id': '/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/resourceGroups/d-azurelm/providers/Microsoft.MachineLearningServices/workspaces/aml-9cxgm0wt/onlineEndpoints/landing-club-3/deployments/landing-club-3',
 'name': 'landing-club-3',
 'type': 'Microsoft.MachineLearningServices/workspaces/onlineEndpoints/deployments',
 'systemData': {'createdBy': 'admin',
  'createdAt': '2022-12-21T19:44:36.984591Z',
  'lastModifiedAt': '2022-12-21T19:44:36.984591Z'},
 'tags': {},
 'location': 'northeurope',
 'kind': 'Managed',
 'properties': {'environmentVariables': {},
  'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/providers/Microsoft.MachineLearningServices/locations/northeurope/mfeOperationsStatus/od:e9441137-e369-431b-aac4-019e675f4683:b7109dee-6761-4bb5-b07a-cd2cd5416053?api-version=2022-02-01-preview'},
  'appInsightsEnabled': False,
  'egressPublicNetworkAccess': 'Enabled',
  'endpointComputeType': 'Managed',
  'i